<a href="https://colab.research.google.com/github/amos-fernandes/machine-learning-bairesdev-dio/blob/dev/Mentoria_DIO_Linguagem_Natural_e_Prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompting, Extraindo a Polpa da Fruta

*Me. Guilherme D. F. Silva, Machine Learning Engineer at [BairesDev](https://www.bairesdev.com)*.

*PhD. Henrique P. Gomide, Machine Learning Engineer at [BairesDev](https://www.bairesdev.com)*.

<br>

Como utilizar diferentes técnicas de prompting para melhorar a performance de LLMs e permitir que elas solucionem problemas mais complexos.

---

Importando bibliotecas e acessando a API [Groq](https://console.groq.com).

In [ ]:
import json
import requests

from google.colab import userdata

In [ ]:
GROQ_KEY = userdata.get("GROQ_KEY")

API_URL = 'https://api.groq.com/openai/v1/chat/completions'
headers = {
    'Authorization': f'Bearer {GROQ_KEY}',
    'Content-Type': 'application/json'
}

---

Um prompt é uma instrução textual usada para guiar um modelo de linguagem (LLM) a gerar uma resposta.

**Tipos de Prompts:**
- User Prompt: Mensagem do usuário que solicita uma ação ou resposta da IA.
    - Ex.: "Explique a teoria da evolução."

- System Prompt: Instruções definidas pelo desenvolvedor para orientar o comportamento do modelo.
    - Ex.: "Seja educado e objetivo nas respostas."

In [ ]:
user_prompt = "Escreva uma estrofe para um poema."
system_prompt = "Você é um cozinheiro que adora falar sobre pizzas."

data = {
    "model": "llama-3.2-3b-preview",
    "messages": [{"role": "user", "content": user_prompt},
                 {"role": "system", "content": system_prompt}],
    "max_tokens": 500
}

response = requests.post(API_URL, headers=headers, json=data)
result = response.json()
print(result["choices"][0]["message"]["content"])

Em fornos quentes e cheios de magia,
Dois elementos se encontram juntos no cielo,
Massa de trigo e fermento, a fusão perfeita,
Encostada a queijo molhado, meu coração está a bater para a pizza deliciosa.


In [ ]:
user_prompt = "Escreva uma estrofe para um poema."
system_prompt = "Você é um matemático."

data = {
    "model": "llama-3.2-3b-preview",
    "messages": [{"role": "user", "content": user_prompt},
                 {"role": "system", "content": system_prompt}],
    "max_tokens": 500
}

response = requests.post(API_URL, headers=headers, json=data)
result = response.json()
print(result["choices"][0]["message"]["content"])

Em número e figuras, encontro harmonia,
Um mundo de matemática, infinito e vasto,
Simetria e cor, num toque de lógica,
Um universo de pureza, onde a razão é a chave.


In [ ]:
user_prompt = "Escreva uma estrofe para um poema."
system_prompt = "Você é um camelo e está em busca de um oasis."

data = {
    "model": "llama-3.2-3b-preview",
    "messages": [{"role": "user", "content": user_prompt},
                 {"role": "system", "content": system_prompt}],
    "max_tokens": 500
}

response = requests.post(API_URL, headers=headers, json=data)
result = response.json()
print(result["choices"][0]["message"]["content"])

"No deserto quente, eu sigo a jornada,
Com passos lentos, e olhos que observam a tarde,
A busca por um sonho, um refúgio do calor,
Um oasis que me leve, em um lugar fresco e mais calmo."


# Como melhorar o desempenho de um modelo usando Prompts?

## Primeiro Problema - Investigando Gírias

O que acontece se apenas pedirmos o significado de uma gíria?

In [ ]:
prompt = "Como um grande conhecedor das gírias brasileiras, me informe qual é o significado da gíria chinelagem"

data = {
    "model": "llama-3.2-3b-preview",
    "messages": [{"role": "user", "content": prompt}],
    "max_tokens": 500
}

response = requests.post(API_URL, headers=headers, json=data)
result = response.json()
print(result["choices"][0]["message"]["content"])

Olá! É um prazer estar aqui para ajudá-lo a explorar a rica paleta de gírias brasileiras.

A gíria "chinelagem" é um termo que se refere a uma combinação de roupas francesas em tons claros e cores vibrantes, frequentemente combinadas com calças de jog Tin e sapatos de madeira ou marfil. No entanto, para mim, essa combinação não faz mais frente para ser definida como chinelagem, após pesquisas por meu conhecimento. Isso ocorre que um termo de chinelagem, trata-se de uma combinação de chinelas de cor e tecido mais específico.


## Formatação de Saída

Também é possível formatar a saída do modelo para que seja como no formato JSON. Para isso, basta solicitarmos ao modelo.

In [ ]:
prompt = (
    "Me diga o significado das 3 seguintes gírias brasileiras: chinelagem, "
    " cusco e bochincho. Por favor, formate a saída em JSON."
)

data = {
    "model": "llama-3.2-3b-preview",
    "messages": [{"role": "user", "content": prompt}],
    "max_tokens": 500
}

response = requests.post(API_URL, headers=headers, json=data)
result = response.json()
print(result["choices"][0]["message"]["content"])

Aqui estão os significados das 3 gírias brasileiras pedidas em formato JSON:

```json
{
  "chinelagem": "Chinelagem é uma gíria brasileira usado para descrever uma pessoa que é considerada preguiçosa, indolente ou desinteressada em atividades.";
  "cusco": "Cusco é uma gíria brasileira usado para se referir a comida, especialmente refeições assadas ou cozidas no forno.";
  "bochincho": "Bochincho é uma gíria brasileira usado para descrever um grupo de pessoas que estão paradas ou se movendo lentamente, muitas vezes de forma desorganizada ou sem propósito específico."
}
```

Espero que isso tenha ajudado! Se tiver mais alguma pergunta, sinta-se à vontade para perguntar.


Quando apenas pedimos uma resposta ao modelo, isso é conhecido como zero-shot prompting. Ou seja, o modelo tenta "acertar com 0 consultas" a exemplos, usando apenas o conhecimento já internalizado na rede neural.

No entanto, se providenciarmos ao modelo alguns exemplos (Few-Shot Prompting), ele pode entender melhor a nossa consulta e também podemos guiar o seu raciocínio.

**Zero-Shot Prompting:**
- Definição: O modelo responde sem exemplos anteriores.
- Exemplo:
    - Input: "Traduza 'gato' para inglês."

**Few-Shot Prompting:**
- Definição: Inclui alguns exemplos no prompt para orientar o modelo.
- Exemplo:
    - Input: "Traduza as palavras seguintes."
    - Exemplos: "gato → cat", "cachorro → dog".
    - Nova palavra: "pássaro → ?"
    - **5-Shot Prompting:** Extensão do few-shot, com 5 exemplos fornecidos.
- Importância: Contexto mais rico melhora a precisão.

In [ ]:
prompt = (
    "Me diga o significado de uma gíria brasileira, siga os exemplos. "
    "Se você não souber o significado da gíria, diga que não sabe, como nos exemplos.\n"
    "Exemplo #1: 'Ancinho' -> 'Eu não sei o significado desta gíria'\n"
    "Exemplo #2: 'Abestado' -> 'Bobo, tolo, lesado'\n"
    "Exemplo #3: 'Sair vazado' -> 'Eu não sei o significado desta gíria'\n"
    "Exemplo #4: 'Firmeza' -> 'Pessoa ou algo positivo'\n"
    "Exemplo #5: 'Crush' -> 'Eu não sei o significado desta gíria'\n"
    "Agora, me diga qual é o significado da gíria: bochincho'"
)

data = {
    "model": "llama-3.2-3b-preview",
    "messages": [{"role": "user", "content": prompt}],
    "max_tokens": 500,
    "temperature": 0.0,
}

response = requests.post(API_URL, headers=headers, json=data)
result = response.json()
print(result["choices"][0]["message"]["content"])

A gíria "bochincho" é um termo que se refere a uma pessoa que é considerada muito chata ou insípida. É como se essa pessoa estivesse "bochinhando" ou perdendo tempo de forma muito lenta e sem interesse.

Exemplo: "Eu não sei por que ele está tão chato, ele é um bochincho!"

É importante notar que o significado de gírias pode variar dependendo do contexto e da região, mas em geral, "bochincho" é usado para descrever alguém que é muito chato ou sem interesse.


In [ ]:
prompt = "Como um grande conhecedor das gírias brasileiras, me informe sucintamente qual é o significado da gíria: purla"

data = {
    "model": "llama-3.2-3b-preview",
    "messages": [{"role": "user", "content": prompt}],
    "max_tokens": 500
}

response = requests.post(API_URL, headers=headers, json=data)
result = response.json()
print(result["choices"][0]["message"]["content"])

Olá!

Eu posso informar sobre a gíria "purla". É uma gíria que vem do inglês "purl", que significa "rodar" ou "lugar de muita atividade". No Brasil, a gíria "purla" é usada para se referir a um lugar de diverções, espetáculos ou festivais, como um circo, um show, um clube noturno, entre outros.

Além disso, a gíria pode ser usada para expressar que existe uma turba ou multidão no local onde está se referindo.

 É um termo coloquial e informal, não é amplamente usado no dia a dia, na vida corrente.


In [ ]:
prompt = (
    "Me diga o significado de uma gíria brasileira, siga os exemplos. "
    "Se você não souber o significado da gíria, diga que não sabe, como nos exemplos.\n"
    "Exemplo #1: 'Ancinho' -> 'Eu não sei o significado desta gíria'\n"
    "Exemplo #2: 'Abestado' -> 'Bobo, tolo, lesado'\n"
    "Exemplo #3: 'Sair vazado' -> 'Eu não sei o significado desta gíria'\n"
    "Exemplo #4: 'Firmeza' -> 'Pessoa ou algo positivo'\n"
    "Exemplo #5: 'Crush' -> 'Eu não sei o significado desta gíria'\n"
    "Agora, me diga qual é o significado da gíria: purla'"
)

data = {
    "model": "llama-3.2-3b-preview",
    "messages": [{"role": "user", "content": prompt}],
    "max_tokens": 500,
    "temperature": 0.1,
}

response = requests.post(API_URL, headers=headers, json=data)
result = response.json()
print(result["choices"][0]["message"]["content"])

A gíria "purla" é um termo que pode ter diferentes significados dependendo do contexto em que é usado. No entanto, é comum associar "purla" a algo ou alguém que é considerado muito bonito ou atraente.

Por exemplo, se alguém disser "Essa pessoa é uma purla!", provavelmente estará expressando admiração ou admiração por sua beleza ou charme.

É importante notar que o significado de "purla" pode variar dependendo da região ou do contexto em que é usado, e pode não ser amplamente reconhecido ou aceito por todos.


## Retrieval Augmented Generation (RAG):

Técnica que combina modelos de linguagem com bases de dados externas.
- Como Funciona:
    - Entrada: O prompt é enviado para o sistema.
    - Recuperação: O modelo busca informações relevantes em uma base externa.
    - Unificação: O prompt é unificado às informações recuperadas e enviado à LLM.
    - Geração: Responde com base na consulta e no contexto recuperado.
- Exemplo:
    - Prompt: "Qual a história da Torre Eiffel?"
    - Embedding: Transforma o prompt em um array de números representando cada palavra.
    - "Qual a história da Torre Eiffel?" → [302, 14, 14912, 30, 3124, 12499]
    - Busca na base de dados por vetores semelhantes ao vetor de prompt.
    - Decodifica o vetor mais semelhante e o apresenta como contexto para o prompt.
    - Resposta: "A Torre Eiffel foi construída em 1889 para a Exposição Universal."
- Importância: Respostas mais precisas e baseadas em fatos, reduz alucinações.


**Nossa base de dados de gírias para o nosso RAG:**

https://conteudo.sesc-rs.com.br/girias-gauchas-quantos-destes-termos-voce-conhece

https://pt.wikipedia.org/wiki/Dialeto_ga%C3%BAcho

https://www.dicionarioinformal.com.br/bochincho


In [ ]:
context = (
    """Cusco: Cachorro sem raça definida
    Resume cão pequeno, vira-lata. Sinônimo de guaipeca. A palavra é usada sozinha ou em expressões populares como frio de renguear cusco (frio insuportável) ou mais perdido que cusco em tiroteio.
    "Amiga, adotei um cusquinho tão lindo."

    Esgualepado: Sem movimento, cansado, exausto
    Define uma pessoa, animal ou objeto que está em condições precárias, cansado, mal cuidado ou danificado. São sinônimos: arrebentado, esfarrapado, esgotado, etc.
    "Tô toda esgualepada, ainda bem que hoje é sexta-feira!"

    Lagartear: Deitar, sentar
    Mais do que o significado formal, no Sul "lagartear" de verdade é curtir a preguiça no sol, especialmente em dias de inverno. Depois do almoço, é uma ótima pedida. Se tiver uma bergamota junto, melhor ainda.

    Bochincho: Gíria de gaúcho. Bebedeira, desordem, briga, bagunça, baile popular; arrasta-pé
    Em muitas peleias no bochincho, aprendi a não pelear com mulher casada
    """
)
prompt = "Usando o contexto apresentado, me diga o que significa a gíria bochincho."

data = {
    "model": "llama-3.2-3b-preview",
    "messages": [
        {"role": "system", "content": context},
        {"role": "user", "content": prompt}
    ],
    "max_tokens": 500
}

response = requests.post(API_URL, headers=headers, json=data)
result = response.json()
print(result["choices"][0]["message"]["content"])

A gíria "bochincho" é um termo usado principalmente no Sul do Brasil. Segundo o contexto apresentado, ela remete a uma situação de:

- Bebedeira
- Desordem
- Briga
- Baile popular
- Arrasta-pé

Em outras palavras, quando mencionado o bochincho, está se referindo a uma situação animada, com muitas pessoas, bebidas, música e talvez briga ou bagunça.


In [ ]:
messages = [
    {"role": "user", "content": "Usando o contexto apresentado, me diga o que significa a gíria bochincho."},
    {"role": "assistant", "content": """De acordo com o contexto apresentado, a gíria "bochincho" refere-se a:

        - Gíria de gaúcho
        - Bebedeira, desordem, briga, bagunça, baile popular; arrasta-pé

        Em outras palavras, "bochincho" é um termo popular que descreve uma situação de caos, atividade social excessiva, dança, música, festa ou briga.
    """},
    {"role": "user", "content": "Me diga o significado das 3 seguintes gírias brasileiras: chinelagem, "
    " cusco e bochincho. Por favor, formate a saída em JSON."}
]

data = {
    "model": "llama-3.2-3b-preview",
    "messages": messages,
    "max_tokens": 500
}

response = requests.post(API_URL, headers=headers, json=data)
result = response.json()
print(result["choices"][0]["message"]["content"])

Aqui estão os significados das 3 gírias brasileiras solicitadas, formatados em JSON:

```
[
  {
    "Gíria": "chinelagem",
    "Significado": "Dance folclórica típica da Amazônia, especialmente realizada no estado do Pará"
  },
  {
    "Gíria": "cusco",
    "Significado": "Alcanfora ou alcaparra"
  },
  {
    "Gíria": "bochincho",
    "Significado": "Bebedeira, desordem, briga, bagunça, baile popular"
  }
]
```

Espero que isso esteja de acordo com suas necessidades.


# Segundo Problema - Chain-of-Thought (CoT)

Chain-of-Thought é uma técnica que encoraja o modelo a "pensar em voz alta" antes de responder.
- Como Funciona:
    - Prompt: "Quantas maçãs restam se você comeu 2 de 5?"
    - Resposta com Chain-of-Thought:
        1. "Eu tinha 5 maçãs."
        2. "Comi 2 maçãs."
        3. "Agora restam 3 maçãs."
        4. Resposta Final: 3.
- Importância: Melhora a precisão em tarefas complexas pela divisão de um problema maior em problemas menores.

In [ ]:
prompt = (
    "Se um trem viaja a 60 km/h durante 2 horas e ainda precisa de meia hora para terminar a viagem, qual é a distância entre a origem e o destino?"
)

data = {
    "model": "llama-3.2-3b-preview",
    "messages": [{"role": "user", "content": prompt}],
    "max_tokens": 500
}

response = requests.post(API_URL, headers=headers, json=data)
result = response.json()
print(result["choices"][0]["message"]["content"])

Para resolver esse problema, precisamos calcular a distância percorrida pelo trem durante as primeiras 2 horas e acrescentar a distância percorrida nos últimos 30 minutos.

Em 1 hora, o trem viaja 60 km. Portanto, em 2 horas, ele viaja 60 km/h + 60 km/h = 120 km.

Além disso, we constatamos que 1 dia tem 24 horas.

No decorrer de 2 horas, a velocidade do trem é de 60km/h. Logo, em 30 minutos ou (1/2 hora), ele viaja (1/2) x 60km/h.


In [ ]:
prompt = (
    "Resolva o seguinte problema de matemática passo a passo:\n"
    "Se um trem viaja a 60 km/h durante 2 horas e ainda precisa de meia hora para terminar a viagem, qual é a distância entre a origem e o destino?"
)

data = {
    "model": "llama-3.2-3b-preview",
    "messages": [{"role": "user", "content": prompt}],
    "max_tokens": 500
}

response = requests.post(API_URL, headers=headers, json=data)
result = response.json()
print(result["choices"][0]["message"]["content"])

Vamos resolver o problema passo a passo:

**Passo 1: Calcule a distância percorrida pelo trem nos 2 primeiros horas**

Distância = Velocidade x Tempo

Distância = 60 km/h x 2 horas

Distância = 120 km

**Passo 2: Calcule a taxa do trem após a segunda hora**

Velocidade fixa é de 60 km/h, então a taxa após a segunda hora permanece a mesma.

**Passo 3: Calcule a distância restante para o destino**

A meia hora é igual a 0,5 hora. Para calcular a distância restante, usamos a fórmula:

Distância = Velocidade x Tempo

Distância = 60 km/h x 0,5 horas

Distância = 30 km

**Passo 4: Calcule a distância total entre o início e o destino**

Distância total = Distância percorrida + Distância restante

Distância total = 120 km + 30 km

Distância total = 150 km

**Resposta final:**

A distância entre a origem e o destino é de 150 km.


# Concluindo

Quanto mais específicos e detalhistas formos em nossos prompts, mais conhecimento já existente no modelo conseguiremos trazer à tona. As técnicas de prompt engineering ajudam o modelo a apresentar seu verdadeiro nível de conhecimento. Logo, extraímos a polpa da fruta 🥝.